<a href="https://colab.research.google.com/github/swap-10/TextSummarizer-Abstractive/blob/main/AbstractiveTextSummarization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%%bash
mkdir ~/.kaggle
cp kaggle.json ~/.kaggle/
chmod 600 ~/.kaggle/kaggle.json

In [ ]:
!pip install -q kaggle

In [ ]:
!kaggle datasets download -d snap/amazon-fine-food-reviews

 96% 233M/242M [00:02<00:00, 156MB/s]
100% 242M/242M [00:02<00:00, 114MB/s]


In [ ]:
!unzip /content/amazon-fine-food-reviews.zip

Archive:  /content/amazon-fine-food-reviews.zip
  inflating: Reviews.csv             
  inflating: database.sqlite         
  inflating: hashes.txt              


In [ ]:
!cp /content/Reviews.csv /content/drive/MyDrive/Datasets/AmazonReviewsSummarization/

# Let's Get Started

## Installing required packages

In [ ]:
%%bash
pip install datasets
pip install transformers
pip install rouge-score

  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [ ]:
import datasets
import transformers
import pandas as pd
from datasets import Dataset

from transformers import RobertaTokenizerFast
from transformers import EncoderDecoderModel
from transformers import TrainingArguments

from transformers.trainer_seq2seq import Seq2SeqTrainer
from transformers.training_args_seq2seq import Seq2SeqTrainingArguments
from dataclasses import dataclass, field as dataclassfield
from typing import Optional

In [ ]:
import pandas as pd
df = pd.read_csv('/content/drive/MyDrive/Datasets/AmazonReviewsSummarization/Reviews.csv')
print("Shape: ", df.shape)
df.columns

Shape:  (568454, 10)


Index(['Id', 'ProductId', 'UserId', 'ProfileName', 'HelpfulnessNumerator',
       'HelpfulnessDenominator', 'Score', 'Time', 'Summary', 'Text'],
      dtype='object')

In [ ]:
df.drop(columns=['Id', 'ProductId', 'UserId', 'ProfileName', 'HelpfulnessNumerator', 'HelpfulnessDenominator', 'Score', 'Time'], axis=1, inplace=True)
df = df.dropna()

In [ ]:
print("Shape:", df.shape)
df.columns

Shape: (568427, 2)


Index(['Summary', 'Text'], dtype='object')

In [ ]:
df.head(10)

,Summary,Text
0,Good Quality Dog Food,I have bought several of the Vitality canned d...
1,Not as Advertised,Product arrived labeled as Jumbo Salted Peanut...
2,"""Delight"" says it all",This is a confection that has been around a fe...
3,Cough Medicine,If you are looking for the secret ingredient i...
4,Great taffy,Great taffy at a great price. There was a wid...
5,Nice Taffy,I got a wild hair for taffy and ordered this f...
6,Great! Just as good as the expensive brands!,This saltwater taffy had great flavors and was...
7,"Wonderful, tasty taffy",This taffy is so good. It is very soft and ch...
8,Yay Barley,Right now I'm mostly just sprouting this so my...
9,Healthy Dog Food,This is a very healthy dog food. Good for thei...


In [ ]:
df = df.sample(frac=0.08).reset_index(drop=True)

In [ ]:
print("Shape:", df.shape)
df.columns

Shape: (45474, 2)


Index(['Summary', 'Text'], dtype='object')

In [ ]:
df.head(10)

,Summary,Text
0,Good for the price,My cats are kind of finicky and do not like a ...
1,Good flavor and above average coffee bean quality,While not as good as millstone columbian supre...
2,Jack Link's is always great,I love the steak nuggets (and tenders) even be...
3,A mixed bag of good and bad (get the pun??),"As with any selection of differing flavors, th..."
4,great product,well u get the energy u really want for the 5 ...
5,Don't waste your money or your Keurig on this!,This coffee tastes very flavorful and is not t...
6,CS English Breakfast Black Tea,This is a tasty tea that can be enjoyed at any...
7,"Coffee tastes good, not great",The coffee is not quite as strong as I had hop...
8,Surgery for the dog after eating this bone !!,We bought this bone for our 2 year old lab and...
9,Wonderful straight out of the bag!,I love this granola! It's not an overpowering ...


In [ ]:
# Converting data to Huggingface dataset

train_df = df.sample(frac=0.95)
train_data = Dataset.from_pandas(train_df)

rem_df = df.drop(train_df.index)
val_df = rem_df.sample(frac=0.5)
val_data = Dataset.from_pandas(val_df)

test_df = rem_df.drop(val_df.index)
test_data = Dataset.from_pandas(test_df)

## Tokenizing the data

In [ ]:
tokenizer = RobertaTokenizerFast.from_pretrained("roberta-base")
tokenizer.bos_token = tokenizer.cls_token
tokenizer.eos_token = tokenizer.sep_token
tokenizer.pad_token = "<pad>"

# hparams

batch_size = 32
encoder_maxlen = 40
decoder_maxlen = 8

Downloading:   0%|          | 0.00/878k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/446k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.29M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/481 [00:00<?, ?B/s]

In [ ]:


def preprocess_data_to_model(batch):

  inputs = tokenizer(batch["Text"], padding="max_length", truncation=True, max_length = encoder_maxlen)
  outputs = tokenizer(batch["Summary"], padding="max_length", truncation=True, max_length=decoder_maxlen)

  batch["input_ids"] = inputs.input_ids
  batch["attention_mask"] = inputs.attention_mask
  batch["decoder_input_ids"] = outputs.input_ids
  batch["decoder_attention_mask"] = outputs.attention_mask
  batch["labels"] = outputs.input_ids.copy()

  # because RoBERTa automatically shifts the labels, the labels correspond exactly to "decoder_input_ids"
  # We have to make sure that the PAD token is ignored
  batch["labels"] = [[-100 if token == tokenizer.pad_token_id else token for token in labels] for labels in batch["labels"]]

  return batch


# preprocess training data

train_data = train_data.map(preprocess_data_to_model,
                            batched=True,
                            batch_size=batch_size,
                            remove_columns=["Text", "Summary"]
                            )

train_data.set_format(
    type="torch", columns=["input_ids", "attention_mask", "decoder_input_ids", "decoder_attention_mask", "labels"],

)

val_data = val_data.map(
    preprocess_data_to_model,
    batched=True,
    batch_size=batch_size,
    remove_columns=["Text", "Summary"]
)

val_data.set_format(
    type="torch", columns=["input_ids", "attention_mask", "decoder_input_ids", "decoder_attention_mask", "labels"]

)


  0%|          | 0/1350 [00:00<?, ?ba/s]

  0%|          | 0/36 [00:00<?, ?ba/s]

In [ ]:
from transformers import EncoderDecoderModel

roberta_shared = EncoderDecoderModel.from_encoder_decoder_pretrained("roberta-base", "roberta-base", tie_encoder_decoder=True)

Downloading:   0%|          | 0.00/478M [00:00<?, ?B/s]

Some weights of the model checkpoint at roberta-base were not used when initializing RobertaModel: ['lm_head.decoder.weight', 'lm_head.bias', 'lm_head.dense.weight', 'lm_head.dense.bias', 'lm_head.layer_norm.bias', 'lm_head.layer_norm.weight']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForCausalLM were not initialized from the model checkpoint at roberta-base and are newly initialized: ['roberta.encoder.layer.4.crossattention.output.dense.bias', 'roberta.encoder.layer.0.crossattention.output.dense.bias', 'roberta.encoder.layer.3.crossattention.self.key.we

In [ ]:
roberta_shared.config.decoder_start_token_id = tokenizer.bos_token_id
roberta_shared.config.eos_token_id = tokenizer.eos_token_id
roberta_shared.config.pad_token_id = 1
# parameters for beam search
# set decoding params
roberta_shared.config.max_length = 40
roberta_shared.config.early_stopping = True
roberta_shared.config.no_repeat_ngram_size = 3
roberta_shared.config.length_penalty = 2.0
roberta_shared.config.num_beams = 4
roberta_shared.config.vocab_size = roberta_shared.config.encoder.vocab_size

## Metric Computation

In [ ]:
# load rouge for validation
rouge = datasets.load_metric("rouge")

def compute_metrics(pred):
  labels_ids = pred.label_ids
  pred_ids = pred.predictions

  # all unnecessary tokens are removed
  pred_str = tokenizer.batch_decode(pred_ids, skip_special_tokens=True)
  labels_ids[labels_ids == -100] = tokenizer.pad_token_id
  label_str = tokenizer.batch_decode(labels_ids, skip_special_tokens=True)

  rouge_output = rouge.compute(predictions=pred_str, references=label_str, rouge_types=["rouge2"])["rouge2"].mid

  return
  {
      "rouge2_precision": round(rouge_output.precision, 4),
      "rouge2_recall": round(rouge_output.recall, 4),
      "rouge2_measure": round(rouge_output.fmeasure, 4),
  }

Downloading:   0%|          | 0.00/2.16k [00:00<?, ?B/s]

## Training the model

In [ ]:
training_args = Seq2SeqTrainingArguments(
    output_dir = "./",
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    predict_with_generate=True,
    evaluation_strategy='epoch',
    do_train=True,
    do_eval=True,
    logging_steps=4,
    save_steps=16,
    eval_steps=500,
    warmup_steps=500,
    overwrite_output_dir=True,
    save_total_limit=1,
    fp16=True,
)

trainer = Seq2SeqTrainer(
    model=roberta_shared,
    args=training_args,
    compute_metrics=compute_metrics,
    train_dataset=train_data,
    eval_dataset=val_data,
)

trainer.train()

Using amp half precision backend
The following columns in the training set  don't have a corresponding argument in `EncoderDecoderModel.forward` and have been ignored: __index_level_0__.
/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:309: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use thePyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
***** Running training *****
  Num examples = 43200
  Num Epochs = 3
  Instantaneous batch size per device = 32
  Total train batch size (w. parallel, distributed & accumulation) = 32
  Gradient Accumulation steps = 1
  Total optimization steps = 4050
/usr/local/lib/python3.7/dist-packages/transformers/models/encoder_decoder/modeling_encoder_decoder.py:531: FutureWarning: Version v4.12.0 introduces a better way to train encoder-decoder models by computing the loss inside the encoder-decoder framework 

Epoch,Training Loss,Validation Loss


Saving model checkpoint to ./checkpoint-16
Configuration saved in ./checkpoint-16/config.json
Model weights saved in ./checkpoint-16/pytorch_model.bin
/usr/local/lib/python3.7/dist-packages/transformers/models/encoder_decoder/modeling_encoder_decoder.py:531: FutureWarning: Version v4.12.0 introduces a better way to train encoder-decoder models by computing the loss inside the encoder-decoder framework rather than in the decoder itself. You may observe training discrepancies if fine-tuning a model trained with versions anterior to 4.12.0. The decoder_input_ids are now created based on the labels, no need to pass them yourself anymore.
  warnings.warn(DEPRECATION_WARNING, FutureWarning)
Saving model checkpoint to ./checkpoint-32
Configuration saved in ./checkpoint-32/config.json
Model weights saved in ./checkpoint-32/pytorch_model.bin
Deleting older checkpoint [checkpoint-16] due to args.save_total_limit
/usr/local/lib/python3.7/dist-packages/transformers/models/encoder_decoder/modeling_

TypeError: ignored

In [ ]:
import gc
gc.collect()

7066

In [ ]:
!cp -r /content/checkpoint-1344 /content/drive/MyDrive/Datasets/AmazonReviewsSummarization/